# Set constants

In [2]:
test_data_path = "./tmp/test.csv"

# Load test data

In [3]:
import yfinance as yf
import pandas as pd
from NNTrade.common.candle_col_name import INDEX, OPEN,CLOSE, HIGH, LOW, VOLUME

In [4]:
if 1==0:
    data_df:pd.DataFrame =  yf.Ticker("EURUSD=X").history(start="2023-01-01",end="2024-01-01", interval="1d")
    data_df.index = data_df.index.date
    data_df.index.name = INDEX
    data_df["High"] = data_df[["Open","High","Low","Close"]].max(axis=1)
    data_df["Low"] = data_df[["Open","High","Low","Close"]].min(axis=1)
    data_df["Volume"] = 10000000
    data_df.rename({"Open":OPEN, "High": HIGH, "Low":LOW, "Close":CLOSE, "Volume":VOLUME},axis=1, inplace=True)
    data_df.to_csv(test_data_path)

In [5]:
data_df:pd.DataFrame = pd.read_csv(test_data_path, index_col=INDEX)
data_df.index = pd.to_datetime(data_df.index)
data_df

,open,high,low,close,volume,Dividends,Stock Splits
start_date_time,,,,,,,
2023-01-02,1.070973,1.071237,1.065326,1.070973,10000000,0.0,0.0
2023-01-03,1.067771,1.068262,1.052155,1.067771,10000000,0.0,0.0
2023-01-04,1.054685,1.063151,1.054596,1.054685,10000000,0.0,0.0
2023-01-05,1.060637,1.063264,1.051558,1.060637,10000000,0.0,0.0
2023-01-06,1.052222,1.062225,1.048526,1.052222,10000000,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-25,1.102657,1.104240,1.099989,1.102657,10000000,0.0,0.0
2023-12-26,1.102026,1.103997,1.100958,1.102026,10000000,0.0,0.0
2023-12-27,1.104301,1.112248,1.102925,1.104301,10000000,0.0,0.0


In [6]:
data_df.index

DatetimeIndex(['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
               '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11',
               '2023-01-12', '2023-01-13',
               ...
               '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21',
               '2023-12-22', '2023-12-25', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='start_date_time', length=260, freq=None)

# Set up data source

In [7]:
data_source = {"main":data_df}

# Set strategy

In [8]:
from pyalgotrade.technical import ma
from src.common.candle_data_set_config import CandleDataSetConfig
from src.simulation.config.strategy_config import StrategyConfig
from src.simulation.realization.pyalgotrade.strategy_wrapper import PyalgotradeStrategyWrapper
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma
from pyalgotrade.bar import BasicBar
from pyalgotrade.strategy.position import LongPosition, Position, ShortPosition
from pyalgotrade.broker import OrderExecutionInfo, Order
from src.simulation.models import Deal

class MyStrategy(PyalgotradeStrategyWrapper):
    def __init__(self, barFeed,  strategy_config: StrategyConfig):
        super().__init__(barFeed, strategy_config)
        # We want a 15 period SMA over the closing prices.
        self.__instrument = "main"
        self.__sma = ma.SMA(barFeed[self.__instrument].getCloseDataSeries(), 15)
        self.__position:Position = None
        self.__last_deal:Deal = None
        self.__signal = float(strategy_config["delta"])

    def _action_to_stirng(self, action):
        if action == Order.Action.BUY:
            return "BUY"
        elif action == Order.Action.SELL:
            return "SELL"
        elif action == Order.Action.SELL_SHORT:
            return "SELL_SHORT"
        elif action == Order.Action.BUY_TO_COVER:
            return "BUY_TO_COVER"
        else:
            raise Exception(f"Unexpected action {action}")

    def onEnterOk(self, position:Position):
        order: Order = position.getEntryOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()

        self.info(f"{self._action_to_stirng(order.getAction())} at {execInfo.getPrice()}")
        self.__last_deal = Deal.BuildFromCap(execInfo.getDateTime(), execInfo.getPrice(), execInfo.getQuantity(), order.getInstrument(), self.__get_cur_equity())
        self._deal_list.append(self.__last_deal)

    def onEnterCanceled(self, position:Position):
        self.__position = None

    def onExitOk(self, position:Position):
        order: Order = position.getExitOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()
        self.info(f"Close {self._action_to_stirng(order.getAction())} order at {execInfo.getPrice()}")
        self.__position = None
        self.__last_deal.close_deal(execInfo.getDateTime(), execInfo.getPrice())
        self.__last_deal = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def __get_cur_equity(self)->float:
        return self.getBroker().getEquity()

    def onBars(self, bars):
        equity = self.__get_cur_equity()

        bar: BasicBar = bars[self.__instrument]
        
        self._cap_log[bar.getDateTime()] = equity

        if self.__sma[-1] is None:
            return
        
        
        if bar.getPrice()/self.__sma[-1] - 1 > self.__signal:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 100000, True)
            elif isinstance(self.__position, ShortPosition) and not self.__position.exitActive():
                self.__position.exitMarket()
        elif self.__sma[-1]/bar.getPrice() - 1 > self.__signal:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterShort(self.__instrument, 100000, True)
            elif isinstance(self.__position, LongPosition) and not self.__position.exitActive():
                self.__position.exitMarket()


# Optimizing

In [9]:
from src.optimization_analyzer.optimization_analyzer import OptimizationAnalyzer, OptimizationConfig
from NNTrade.common import TimeFrame

In [10]:
from datetime import date
from src.common.candle_config import CandleConfig
from src.common.candle_data_set_config import CandleDataSetConfig
from src.common.date_period import DatePeriod
from src.optimization.config.strategy_config_sets import StrategyConfigSet


opt_config = OptimizationConfig(
    CandleDataSetConfig.BuildFrom(CandleConfig("EURUSD",TimeFrame.D),TimeFrame.D,"main"), 
    DatePeriod(date(2023,1,1), date(2024,1,1)),
    StrategyConfigSet({"delta":[0,0.0001,0.0005, 0.0010, 0.005, 0.01,]}))

In [11]:
from src.simulation.realization.pyalgotrade.pyalgotrade_trading_simulator import PyalgotradeTradingSimulator,Frequency


trading_simulator = PyalgotradeTradingSimulator(data_source, lambda bf, str_cfg:MyStrategy(bf,str_cfg))

In [12]:
opt_analyzer = OptimizationAnalyzer(trading_simulator)
analyzation_report_arr = opt_analyzer.analis_optimization_flow(opt_config,False)
analyzation_report_arr

2023-01-23 00:00:00 strategy [INFO] BUY at 1.0864723920822144
2023-02-08 00:00:00 strategy [INFO] Close SELL order at 1.0729728937149048
2023-02-09 00:00:00 strategy [INFO] SELL_SHORT at 1.0715816020965576
2023-03-15 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.0727657079696655
2023-03-23 00:00:00 strategy [INFO] BUY at 1.0870392322540283
2023-05-16 00:00:00 strategy [INFO] Close SELL order at 1.0873701572418213
2023-05-17 00:00:00 strategy [INFO] SELL_SHORT at 1.0865668058395386
2023-06-19 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.0942115783691406
2023-06-20 00:00:00 strategy [INFO] BUY at 1.0924304723739624
2024-03-02 12:25:28,433 absTradingSimulatior[{'name': 'Example Strategy', 'v': 'v1'}] [INFO] Convert log into report
2024-03-02 12:25:28,433 absTradingSimulatior[{'name': 'Example Strategy', 'v': 'v1'}] [INFO] Simulation report metric:
{'capital': {'max_fall': -0.0038659884469696044,
             'max_yield': 0.0014812827110290971,
             'yield': -0.0

[AnalyzationReport(optimization={'abs_cap_log': {datetime.datetime(2023, 1, 2, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 3, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 4, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 5, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 6, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 9, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 10, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 11, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 12, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 13, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 16, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 17, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 18, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 19, 0, 0): 1000000,
                  datetime.datetime(2023, 1, 20, 0, 0): 1000000,
